<a href="https://colab.research.google.com/github/habiclaude88/Chatbot/blob/main/arnaud_haystack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

nvidia-smi

Wed Dec  7 12:13:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%bash

pip install --upgrade pip  #upgrading the pip installer
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] 
#Clone the Haystack Repo locally

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

#Store the text on the RAM Memory
document_store = InMemoryDocumentStore()

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


#Fetch the dataset
doc_dir = "data/tutorial3" #Where to store the data locally path
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)



INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip to 'data/tutorial3'


True

In [ ]:
#Get a new dataset
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/Ngabo-bajo/NLP-FELLOWSHIP/main/jobinRwanda.csv')

In [ ]:
df.iloc[1]['Job Full Info'].split('\n')

['',
 'AVIS D’APPEL D’OFFRE',
 'REFERENCE : No\xa006/RIM LTD/2022',
 'TITRE DE MARCHE\xa0: CONSTRUCTION DU GUICHET DE BYUMBA',
 'SOURCE DE FINANCEMENT : BUDGET ORDINAIRE',
 'Réseau\xa0 Interdiocésain de Microfinance, R.I.M en sigle, est une institution de Microfinance créée le\xa0\xa0\xa0\xa0\xa0\xa0 06 mai 2004 par l’Eglise Catholique du\xa0 Rwanda et agréée par la BNR le 04 Octobre 2004.',
 'RIM a 11 agences et 36 guichets. Il exerce ses activités dans presque tout le pays et ses guichets\xa0 ont été ouverts dans\xa0 30 Districts du pays. RIM LTD a pour mission de contribuer à la réduction de la pauvreté par les activités d’épargne et de crédit, contribuer à l’essor de la croissance économique du pays par le renforcement de l’esprit d’entreprise chez la population rwandaise économiquement active et promouvoir la dignité humaine.',
 'RIM Ltd lance un appel d’offre public pour la construction du guichet de BYUMBA. Les dossiers d’appel d’offre peuvent être retirés au Siège du RIM Ltd pe

In [ ]:
job_dir = 'jobs'

df_len = len(df) #Counts of rows in the Dataframe
for index in range(df_len):
  with open(f'{job_dir}/job_{index}.txt', 'w+') as file:
    file.writelines(df.iloc[index]['Job Full Info'].split('.'))

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


#Download the data and clean it
docs = convert_files_to_docs(dir_path=job_dir, clean_func=clean_wiki_text, split_paragraphs=True)

In [ ]:
#Store the data into the database
document_store.write_documents(docs)

In [ ]:
# An in-memory TfidfRetriever based on Pandas dataframes
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

#Download and initiate the model
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [ ]:
#Bundle together the model and retriver(Which is the extractor)
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
df.head()

,Hiring Company,Category,Job Position,Job Link,Sector,Education Level,Desired Experience,Contract Type,Number of Positions,Location,Job Views,Job Full Info,Other Company Jobs link
0,Development Bank of Rwanda (BRD),Consultancy,Hiring a Firm for BRD Maintenance Assessment,https://www.jobinrwanda.com/job/hiring-firm-brd-maintenance-assessment,Other,Not specified,Not specified,Contract,1,RW,122,\nRE-ADVERTISEMENT OF TENDER : Nº: 022/10/2022/BRD/HC&CS/CS\nASSIGNMENT TITL...,https://www.jobinrwanda.com/employer/development-bank-rwanda-brd
1,Reseau Interdiocesain de Microfinance( RIM Ltd),Tender,Construction du Guichet de Byumba,https://www.jobinrwanda.com/job/construction-du-guichet-de-byumba,Other,Not specified,Not specified,Tender,1,RW,239,\nAVIS D’APPEL D’OFFRE\nREFERENCE : No 06/RIM LTD/2022\nTITRE DE MARCHE : CO...,https://www.jobinrwanda.com/employer/reseau-interdiocesain-de-microfinance-r...
2,Development Bank of Rwanda (BRD),Consultancy,Consultant- Monitoring & Evaluation,https://www.jobinrwanda.com/job/consultant-monitoring-evaluation-0,Other,Not specified,Not specified,Contract,1,RW,441,\nRE-ADVERTISEMENT OF TENDER : RFP No. 017/08/2022/BRD/HC&CS/CS\nASSIGNMENT ...,https://www.jobinrwanda.com/employer/development-bank-rwanda-brd
3,GardaWorld,Job,Operations Manager,https://www.jobinrwanda.com/job/operations-manager-6,"Administration, Business, Management, Other",Bachelor,Senior (5+ years of experience),Full-time,1,RW,2095,\nJob Description – Operations Manager\n\nPosition/Employment type:\n\n\n Op...,https://www.jobinrwanda.com/employer/gardaworld
4,UN Women Rwanda,Consultancy,Terms of Reference for Production Of the Second Generation of the Country Ge...,https://www.jobinrwanda.com/job/terms-reference-production-second-generation...,"Economics, Other, Social sciences",Master,Senior (5+ years of experience),Contract,1,RW,614,\nTERMS OF REFERENCE FOR PRODUCTION OF THE SECOND GENERATION OF THE COUNTRY ...,https://www.jobinrwanda.com/employer/un-women-rwanda


In [ ]:
#Ask the question
prediction = pipe.run(
    query=question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 10}}
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]

In [ ]:
if prediction['answers'][0].to_dict()['score'] <= 0.6:
  print('No Answer')
  answer = 'No Answer'
else:
  print(prediction['answers'][0].to_dict()['answer'])
  answer = prediction['answers'][0].to_dict()['answer']

No Answer


In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Country Gender Status Report', 'type': 'extractive', 'score': 0.11148339509963989, 'context': 'on with GMO\nPay costs related to the assignment to produce a Country Gender Status Report as will be agreed upon by both parties\nHire an individual ex', 'offsets_in_document': [{'start': 9799, 'end': 9827}], 'offsets_in_context': [{'start': 61, 'end': 89}], 'document_id': '45d2da61f9ae42c7ee748975af78c8c5', 'meta': {'name': 'job_4.txt'}}>,
             <Answer {'answer': 'Country Gender Status Report', 'type': 'extractive', 'score': 0.0882200300693512, 'context': 'lars as enshrined in the report\nThe Second generation of the Country Gender Status Report development will cover the period of 2018 to date The report', 'offsets_in_document': [{'start': 6944, 'end': 6972}], 'offsets_in_context': [{'start': 61, 'end': 89}], 'document_id': '45d2da61f9ae42c7ee748975af78c8c5', 'meta': {'name': 'job_4.txt'}}>,
             <Answer {'answer': 'forchildhoodorg', 'type':

In [ ]:
def get_answer(q):
  prediction = pipe.run(
      query=question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}}
  )
  return prediction

In [ ]:
%%bash
pip install -U easynmt

In [ ]:
question

'What is the complete form of FRG?'

In [ ]:
from easynmt import EasyNMT

model = EasyNMT('OPUS-MT')

question = model.translate(raw_query, source_lang=lang, target_lang='en')

answer = model.translate(answer, source_lang='en', target_lang=lang)

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(answer)

Keine Antwort


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langdetect import detect

raw_query = "Was ist die vollständige Form von BRD?"
lang = detect(raw_query)
print(lang)

de


In [ ]:
question

'What is the minimum experience required at the BRD?'